In [31]:
import pandas as pd
import spacy
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [32]:
train.head(10)
print(train.shape)
# test.head(10)
train = train.fillna(" ")
test = test.fillna(" ")
print(train.shape)
train.head(10)

(8348, 4)
(8348, 4)


,id,subject,email,spam
0,0,Subject: A&L Daily to be auctioned in bankrupt...,URL: http://boingboing.net/#85534171\n Date: N...,0
1,1,"Subject: Wired: ""Stronger ties between ISPs an...",URL: http://scriptingnews.userland.com/backiss...,0
2,2,Subject: It's just too small ...,<HTML>\n <HEAD>\n </HEAD>\n <BODY>\n <FONT SIZ...,1
3,3,Subject: liberal defnitions\n,Depends on how much over spending vs. how much...,0
4,4,Subject: RE: [ILUG] Newbie seeks advice - Suse...,hehe sorry but if you hit caps lock twice the ...,0
5,5,Subject: Five\n,URL: http://diveintomark.org/archives/2002/10/...,0
6,6,Subject: [SAdev] [Bug 1006] Spamassassin's bui...,http://www.hughes-family.org/bugzilla/show_bug...,0
7,7,Subject: One of a kind Money maker! Try it for...,------000000000000000000000\n Content-Type: te...,1
8,8,Subject: freshrpms.net resources (was Re: use ...,"Once upon a time, Brian wrote :\n \n > Yeah...",0
9,9,Subject: use of base image / delta image for a...,"Hi,\n \n I was inspired by a mode of operation...",0


In [33]:
train['all'] = train['subject']+train['email']
train['label'] = [{"spam": bool(y), "ham": not bool(y)}
                  for y in train.spam.values]
train.head(10)
# print(train.loc[0]['all'])

,id,subject,email,spam,all,label
0,0,Subject: A&L Daily to be auctioned in bankrupt...,URL: http://boingboing.net/#85534171\n Date: N...,0,Subject: A&L Daily to be auctioned in bankrupt...,"{'spam': False, 'ham': True}"
1,1,"Subject: Wired: ""Stronger ties between ISPs an...",URL: http://scriptingnews.userland.com/backiss...,0,"Subject: Wired: ""Stronger ties between ISPs an...","{'spam': False, 'ham': True}"
2,2,Subject: It's just too small ...,<HTML>\n <HEAD>\n </HEAD>\n <BODY>\n <FONT SIZ...,1,Subject: It's just too small ...,"{'spam': True, 'ham': False}"
3,3,Subject: liberal defnitions\n,Depends on how much over spending vs. how much...,0,Subject: liberal defnitions\nDepends on how mu...,"{'spam': False, 'ham': True}"
4,4,Subject: RE: [ILUG] Newbie seeks advice - Suse...,hehe sorry but if you hit caps lock twice the ...,0,Subject: RE: [ILUG] Newbie seeks advice - Suse...,"{'spam': False, 'ham': True}"
5,5,Subject: Five\n,URL: http://diveintomark.org/archives/2002/10/...,0,Subject: Five\nURL: http://diveintomark.org/ar...,"{'spam': False, 'ham': True}"
6,6,Subject: [SAdev] [Bug 1006] Spamassassin's bui...,http://www.hughes-family.org/bugzilla/show_bug...,0,Subject: [SAdev] [Bug 1006] Spamassassin's bui...,"{'spam': False, 'ham': True}"
7,7,Subject: One of a kind Money maker! Try it for...,------000000000000000000000\n Content-Type: te...,1,Subject: One of a kind Money maker! Try it for...,"{'spam': True, 'ham': False}"
8,8,Subject: freshrpms.net resources (was Re: use ...,"Once upon a time, Brian wrote :\n \n > Yeah...",0,Subject: freshrpms.net resources (was Re: use ...,"{'spam': False, 'ham': True}"
9,9,Subject: use of base image / delta image for a...,"Hi,\n \n I was inspired by a mode of operation...",0,Subject: use of base image / delta image for a...,"{'spam': False, 'ham': True}"


In [34]:
from sklearn.model_selection import StratifiedShuffleSplit
# help(StratifiedShuffleSplit)
splt = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
for train_idx, valid_idx in splt.split(train, train['spam']): 
											# 按照后者分层抽样
    train_set = train.iloc[train_idx]
    valid_set = train.iloc[valid_idx]

# 查看分布
print(train_set['spam'].value_counts()/len(train_set))
print(valid_set['spam'].value_counts()/len(valid_set))


0    0.743636
1    0.256364
Name: spam, dtype: float64
0    0.743713
1    0.256287
Name: spam, dtype: float64


In [35]:
nlp = spacy.blank('en')
email_cat = nlp.create_pipe('textcat', 
#                             config=
#                             {
#     "exclusive_classes": True, # 排他的，二分类
#     "architecture": "bow"
#                             }
                           )
help(nlp.create_pipe)

Help on method create_pipe in module spacy.language:

create_pipe(name, config={}) method of spacy.lang.en.English instance
    Create a pipeline component from a factory.
    
    name (unicode): Factory name to look up in `Language.factories`.
    config (dict): Configuration parameters to initialise component.
    RETURNS (callable): Pipeline component.
    
    DOCS: https://spacy.io/api/language#create_pipe



In [36]:
nlp.add_pipe(email_cat)


In [37]:
email_cat.add_label('ham')
email_cat.add_label('spam')

1

In [38]:
from spacy.util import minibatch
import random
def train(model, train, optimizer, batch_size=8):
    loss = {}
    random.seed(1)
    random.shuffle(train)
    batches = minibatch(train, size=batch_size)
    for batch in batches:
        text, label = zip(*batch)
        model.update(text, label, sgd=optimizer, losses=loss)
    return loss

In [39]:
def predict(model, text):
    docs = [model.tokenizer(txt) for txt in text]
    emailpred = model.get_pipe('textcat')
    score, _ = emailpred.predict(docs)
    pred_label = score.argmax(axis=1)
    return pred_label

In [40]:
def evaluate(model, text, label):
    pred = predict(model, text)
    true_class = [int(lab['cats']['spam']) for lab in label]
    correct = (pred == true_class)
    acc = sum(correct)/len(correct)
    return acc

In [41]:
train_text = train_set['all'].values
train_label = train_set['label']
valid_text = valid_set['all'].values
valid_label = valid_set['label']

train_label = [{"cats": label} for label in train_label]
valid_label = [{"cats": label} for label in valid_label]

train_data = list(zip(train_text, train_label))
# valid_data = zip(valid_text, valid_label)

test_text = (test['subject']+test['email']).values

In [42]:
n = 20
opt = nlp.begin_training()
for i in range(n):
    loss = train(nlp, train_data, opt)
    acc = evaluate(nlp, valid_text, valid_label)
    print(f"Loss: {loss['textcat']:.3f} \t Accuracy: {acc:.3f}")

Loss: 1.132 	 Accuracy: 0.941
Loss: 0.283 	 Accuracy: 0.988
Loss: 0.121 	 Accuracy: 0.993
Loss: 0.137 	 Accuracy: 0.993
Loss: 0.094 	 Accuracy: 0.982
Loss: 0.069 	 Accuracy: 0.995
Loss: 0.060 	 Accuracy: 0.990
Loss: 0.010 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.992
Loss: 0.004 	 Accuracy: 0.991
Loss: 0.004 	 Accuracy: 0.991
Loss: 0.308 	 Accuracy: 0.981
Loss: 0.158 	 Accuracy: 0.987
Loss: 0.014 	 Accuracy: 0.990
Loss: 0.007 	 Accuracy: 0.990
Loss: 0.043 	 Accuracy: 0.990


In [43]:
pred = predict(nlp, test_text)

In [44]:
print(pred)

[0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0
 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0
 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0
 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 1 1 1 1 0 0 0 0 1 0 0 0 

In [45]:
# pred = (pred == 0)
# predi = [int(p) for p in pred]
# print(predi)

In [46]:
id = test['id']
output = pd.DataFrame({'id':id, 'Class':pred})
output.to_csv("submission.csv",  index=False)